# Contexte du projet – Projet 10
___
**Objectif** : développer une première version d’un système de recommandation de contenu pour l’application My Content.
___
La startup My Content souhaite encourager la lecture en recommandant des articles pertinents à ses utilisateurs. Dans cette phase de MVP (Minimum Viable Product), l’objectif est de mettre en place :

un système de recommandation efficace (filtrage collaboratif et/ou basé contenu),

une application simple permettant d’afficher les recommandations,

une architecture déployable sur Azure via une fonction serverless.

Les données utilisées sont publiques et simulées, représentant des interactions entre des utilisateurs et des articles.

In [ ]:
############################
# Importation des librairies
############################

# Manipulation de données
import pandas as pd
import numpy as np

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# Système de fichiers
import os
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"]      = "1"

# Affichage dans Jupyter
from IPython.display import display

# fichier des fonctions
import fonctions as fc
from fonctions import split_temporal, train_test_svd_temporal


In [ ]:
# Hyper-paramètres
CUT_OFF          = None            # None pour quantile 0.8 automatique
PERCENTILE_Q     = 0.8             # utilisé si CUT_OFF=None
K_RECS           = 5               # Top-K recommandations
ALPHA_HYBRIDE    = 0.7             # mix content/collab dans hybrid

In [ ]:
# Définir le chemin vers les données
DATA_PATH = "../data/"

# Chargement des fichiers CSV
articles = pd.read_csv(os.path.join(DATA_PATH, "articles_metadata.csv"))
clicks_sample = pd.read_csv(os.path.join(DATA_PATH, "clicks_sample.csv"))

## Explorations des données

In [ ]:
# Aperçu des 5 premières lignes
print("Aperçu des articles :")
display(articles.head())

print("\nAperçu des interactions (clicks_sample) :")
display(clicks_sample.head())

In [ ]:
# Dimension des datasets
print(f"Articles : {articles.shape[0]} lignes, {articles.shape[1]} colonnes")
print(f"Clicks sample : {clicks_sample.shape[0]} lignes, {clicks_sample.shape[1]} colonnes")

In [ ]:
# Liste des colonnes disponibles
print("Colonnes dans articles:")
print(articles.columns.tolist())

print("\nColonnes dans clicks_sample:")
print(clicks_sample.columns.tolist())

In [ ]:
# Types de données
print("Types de données - Articles :")
print(articles.dtypes)

print("\nTypes de données - Clicks sample :")
print(clicks_sample.dtypes)

In [ ]:
# Vérification des valeurs manquantes
print("Valeurs manquantes - Articles :")
display(articles.isnull().sum())

print("\nValeurs manquantes - Clicks sample :")
display(clicks_sample.isnull().sum())

In [ ]:
# Statistiques descriptives
print("Statistiques - Articles :")
display(articles.describe())

print("\nStatistiques - Clicks sample :")
display(clicks_sample.describe())

## Analyse Univarié
- Objectif : décrire les variables indépendamment

___
### articles_metadata.csv
___

In [ ]:
# Nombre d’articles par "category_id"
plt.figure(figsize=(10, 5))
sns.countplot(data=articles, x="category_id", order=articles["category_id"].value_counts().index)
plt.title("Nombre d'articles par catégorie")
plt.xlabel("Catégorie")
plt.ylabel("Nombre d'articles")
plt.xticks(rotation=45) # reste ilisible
plt.tight_layout()
plt.show()

In [ ]:
# Nombre d’articles par "publisher_id"
plt.figure(figsize=(10, 5))
sns.countplot(data=articles, x="publisher_id", order=articles["publisher_id"].value_counts().index)
plt.title("Nombre d'articles par éditeur")
plt.xlabel("Éditeur (Publisher ID)")
plt.ylabel("Nombre d'articles")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Distribution du nombre de mots par article (words_count)
plt.figure(figsize=(8, 5))
sns.histplot(articles["words_count"], bins=30, kde=True)
plt.title("Distribution du nombre de mots par article")
plt.xlabel("Nombre de mots")
plt.ylabel("Nombre d'articles")
plt.tight_layout()
plt.show()

___
### clicks_sample.csv
___

In [ ]:
# Nombre de clics par utilisateur "user_id"
user_click_counts = clicks_sample["user_id"].value_counts()

plt.figure(figsize=(8, 5))
sns.histplot(user_click_counts, bins=30, kde=False)
plt.title("Nombre de clics par utilisateur")
plt.xlabel("Nombre de clics")
plt.ylabel("Nombre d’utilisateurs")
plt.tight_layout()
plt.show()

In [ ]:
# Nombre de clics par article "click_article_id"
article_click_counts = clicks_sample["click_article_id"].value_counts()

plt.figure(figsize=(8, 5))
sns.histplot(article_click_counts, bins=30, kde=False)
plt.title("Nombre de clics par article")
plt.xlabel("Nombre de clics")
plt.ylabel("Nombre d’articles")
plt.tight_layout()
plt.show()

In [ ]:
# Articles les plus populaires (Top 10)
top_articles = article_click_counts.head(10)
top_articles = top_articles.reset_index()
top_articles.columns = ["article_id", "nb_clicks"]

plt.figure(figsize=(10, 5))
sns.barplot(data=top_articles, x="article_id", y="nb_clicks")
plt.title("Top 10 des articles les plus cliqués")
plt.xlabel("ID Article")
plt.ylabel("Nombre de clics")
plt.tight_layout()
plt.show()

## Analyse multivarié
- Objectif : croiser les variables

___
### Fusion des deux jeux de données
___

In [ ]:
df = clicks_sample.merge(articles, left_on="click_article_id", right_on="article_id", how="left")
print("Dimensions du DataFrame fusionné :", df.shape)
display(df.head(20))

In [ ]:
df.info()

In [ ]:
# Nombre moyen de clics par catégorie d’article
cat_clicks = df.groupby("category_id")["click_article_id"].count().sort_values(ascending=False)

plt.figure(figsize=(10, 5))
sns.barplot(x=cat_clicks.index.astype(str), y=cat_clicks.values)
plt.title("Nombre de clics par catégorie d'article")
plt.xlabel("Catégorie")
plt.ylabel("Nombre de clics")
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
# Nombre moyen de clics par éditeur "publisher_id"
pub_clicks = df.groupby("publisher_id")["click_article_id"].count().sort_values(ascending=False)

plt.figure(figsize=(10, 5))
sns.barplot(x=pub_clicks.index.astype(str), y=pub_clicks.values)
plt.title("Nombre de clics par éditeur")
plt.xlabel("Publisher ID")
plt.ylabel("Nombre de clics")
plt.tight_layout()
plt.show()

In [ ]:
# Moyenne des mots par article selon la catégorie
words_per_category = df.groupby("category_id")["words_count"].mean().sort_values(ascending=False)

plt.figure(figsize=(10, 5))
sns.barplot(x=words_per_category.index.astype(str), y=words_per_category.values)
plt.title("Nombre moyen de mots par catégorie")
plt.xlabel("Catégorie")
plt.ylabel("Mots moyens par article")
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
# Création d’une mini-matrice (échantillon de 50x50) pour visualisation
pivot = df.pivot_table(index="user_id", columns="click_article_id", aggfunc="size", fill_value=0)

plt.figure(figsize=(12, 8))
sns.heatmap(pivot.iloc[:50, :50], cmap="viridis")
plt.title("Interactions User-Article (échantillon 50x50)")
plt.xlabel("Article ID")
plt.ylabel("User ID")
plt.tight_layout()
plt.show()

In [ ]:
# Nombre d’utilisateurs uniques par article
user_per_article = df.groupby("click_article_id")["user_id"].nunique()

plt.figure(figsize=(8, 5))
sns.histplot(user_per_article, bins=30, kde=False)
plt.title("Nombre d’utilisateurs uniques par article")
plt.xlabel("Utilisateurs")
plt.ylabel("Articles")
plt.tight_layout()
plt.show()

In [ ]:
# Nombre d’articles uniques par utilisateur
articles_per_user = df.groupby("user_id")["click_article_id"].nunique()

plt.figure(figsize=(8, 5))
sns.histplot(articles_per_user, bins=30, kde=False)
plt.title("Nombre d’articles uniques par utilisateur")
plt.xlabel("Articles")
plt.ylabel("Utilisateurs")
plt.tight_layout()
plt.show()

In [ ]:
# Corrélations possibles entre words_count, category_id, publisher_id
plt.figure(figsize=(6, 5))
sns.boxplot(data=df, x="category_id", y="words_count")
plt.title("Répartition des words_count par catégorie")
plt.xlabel("Catégorie")
plt.ylabel("Nombre de mots")
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
# ─── Conversion des timestamps (ms) ────────────────────
df["created_at"]    = pd.to_datetime(df["created_at_ts"], unit="ms")
df["click_time"]    = pd.to_datetime(df["click_timestamp"], unit="ms")
df["session_start"] = pd.to_datetime(df["session_start"],   unit="ms")

In [ ]:
# ─── Choix du cutoff pour le split temporel ────────────
# Vous pouvez fixer une date précise :
# cutoff = pd.Timestamp("2017-10-01")
# — ou laisser None pour prendre automatiquement le 80e percentile
cutoff = None

## Modélisation et recommandations

### user-based content-based (Recommandations user_id)

In [ ]:
# Liste des colonnes disponibles
print(df.columns.tolist())

#### CountVectorizer

In [ ]:
# Créer une "signature textuelle" pour chaque article
df["article_features"] = (
    "cat_" + df["category_id"].astype(str) + " " +
    "pub_" + df["publisher_id"].astype(str)
)

# Ne garder qu’un seul vecteur par article
df_articles = df.drop_duplicates(subset="article_id")[["article_id", "article_features"]].copy()

In [ ]:
# Vectorisation avec CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df_articles["article_features"])

# Matrice de similarité cosinus (articles x articles)
similarity_matrix = cosine_similarity(X)

In [ ]:
# Construction de la matrice de similarité
df_articles, similarity_matrix = fc.construire_matrice_similarite(df)

# Test d'une recommandation
user_test = df["user_id"].sample(1).iloc[0]
recommandations = fc.recommender_content_based(user_test, df, df_articles, similarity_matrix)
print(f"Recommandations pour l'utilisateur {user_test} : {recommandations}")

#### Librairie Surprise avec SVD
___
**Objectif** :
- Modéliser les interactions user_id / click_article_id comme des notes implicites (clic = 1)
- Entraîner un modèle avec la librairie surprise
- Générer les recommandations

In [ ]:
# Préparer les données
modele_svd, trainset, testset, metrics = train_test_svd_temporal(
    df, ts_col="created_at_ts", cutoff=CUT_OFF
)
print(f"SVD temporel → RMSE: {metrics['rmse']:.4f}  MAE: {metrics['mae']:.4f}")

# Tester la recommandation
user_test = df["user_id"].sample(1).iloc[0]
recommandations = fc.recommender_surprise(user_test, modele_svd, df)
print(f"Recommandations (Surprise SVD) pour l'utilisateur {user_test} : {recommandations}")

#### Librairie Implicit avec ALS
___
**Objectif** :
- Modéliser les interactions utilisateur ↔ article via une matrice creuse CSR
- Utiliser les clics comme feedback implicites (score = 1 ou pondéré)
- Entraîner un modèle AlternatingLeastSquares (implicit.als)
- Générer les recommandations personnalisées par utilisateur

In [ ]:
# ─── ALS – leave-one-out & évaluation ───────────────────────────────
from fonctions import split_leave_one_out
import numpy as np
from scipy.sparse import csr_matrix
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, ndcg_at_k

# 0) Leave-one-out split
df_train, df_test = split_leave_one_out(df, ts_col="created_at_ts")
print(f"Train interactions : {len(df_train)} — Test interactions : {len(df_test)}")

# 1) Score implicite
df_train["popularity_indicator"] = 1
df_test ["popularity_indicator"] = 1

# 2) CSR sur df_train
csr_train, user_map, item_map, _ = fc.preparer_matrices_implicit(df_train)

# 3) CSR sur df_test (users/items vus en train)
df_test_filt = df_test[
    df_test.user_id.isin(user_map) & df_test.click_article_id.isin(item_map)
].copy()
df_test_filt["user_idx"] = df_test_filt["user_id"].map(user_map)
df_test_filt["item_idx"] = df_test_filt["click_article_id"].map(item_map)
csr_test = csr_matrix(
    (df_test_filt["popularity_indicator"],
    (df_test_filt["user_idx"], df_test_filt["item_idx"])),
    shape=csr_train.shape
)

# 4) Entraînement ALS
als_model = fc.entrainer_modele_als(csr_train)

# 5) Filtrer les utilisateurs “valides” pour éviter ZeroDivisionError
train_users = np.unique(csr_train.nonzero()[0])
test_users  = np.unique(csr_test.nonzero()[0])
valid_users = np.intersect1d(train_users, test_users)
csr_train_eval = csr_train[valid_users, :].tocsr()
csr_test_eval  = csr_test[valid_users, :].tocsr()

# 6) Évaluation (automatic → fallback manuel)
try:
    prec  = precision_at_k   (als_model, csr_train_eval, csr_test_eval, K=K_RECS, num_threads=1)
    map5  = mean_average_precision_at_k(als_model, csr_train_eval, csr_test_eval, K=K_RECS, num_threads=1)
    ndcg5 = ndcg_at_k        (als_model, csr_train_eval, csr_test_eval, K=K_RECS, num_threads=1)
    print(f"ALS leave-one-out → Precision@{K_RECS}: {prec:.4f}  MAP@{K_RECS}: {map5:.4f}  NDCG@{K_RECS}: {ndcg5:.4f}")
except ZeroDivisionError:
    print("⚠️ implicit.evaluation a échoué, on utilise l’évaluation manuelle")
    fc.evaluer_modele_als_manuel(
        model=als_model,
        df_train=df_train,
        df_test=df_test,
        user_map=user_map,
        item_map=item_map,
        csr_train=csr_train,
        k=K_RECS
    )

# 7) (Optionnel) Top-N recommandations sur df_train
user_test = df_train["user_id"].sample(1).iloc[0]
recs = fc.recommander_implicit(user_test, df_train, als_model, user_map, item_map, csr_train)
print(f"Recommandations ALS (leave-one-out) pour {user_test} : {recs}")

In [ ]:
resultats_comparaison = pd.DataFrame([
    {"Modèle": "Content-Based", "Type": "Filtrage contenu", "Métrique": "Precision@5", "Score": "-"},
    {"Modèle": "Content-Based", "Type": "Filtrage contenu", "Métrique": "MAP@5", "Score": "-"},
    {"Modèle": "Content-Based", "Type": "Filtrage contenu", "Métrique": "NDCG@5", "Score": "-"},

    {"Modèle": "SVD (Surprise)", "Type": "Collaboratif", "Métrique": "RMSE", "Score": 0.0465},
    {"Modèle": "SVD (Surprise)", "Type": "Collaboratif", "Métrique": "MAE", "Score": 0.0234},

    {"Modèle": "ALS (Implicit)", "Type": "Collaboratif", "Métrique": "Precision@5", "Score": 0.0086},
    {"Modèle": "ALS (Implicit)", "Type": "Collaboratif", "Métrique": "MAP@5", "Score": 0.0260},
    {"Modèle": "ALS (Implicit)", "Type": "Collaboratif", "Métrique": "NDCG@5", "Score": 0.0284},
])

display(resultats_comparaison)

**Analyse comparative des modèles de recommandation**

- Content-Based
Ce modèle repose uniquement sur les métadonnées des articles. Il n’utilise pas l’historique utilisateur, ce qui rend l’évaluation quantitative difficile ici (aucune métrique calculée directement).
→ Avantage : utile en cold-start, pour les nouveaux utilisateurs ou articles.

- SVD (Surprise)
Le modèle SVD obtient les meilleurs scores d'erreur absolue (RMSE = 0.0465, MAE = 0.0234).
Cela indique une bonne capacité à approximer les "notes implicites" (clics = 1) sur des articles non vus.
→ Il est performant sur des interactions bien représentées et faciles à factoriser.

- ALS (Implicit)
Le modèle ALS donne des scores faibles sur Precision@5, MAP@5 et NDCG@5.    
Cela s’explique par :

    * un dataset très petit (1 883 lignes),

    * des clics binaires peu informatifs (pas de pondération),

    * une faible densité des interactions.
    → Malgré cela, ALS reste utile sur des jeux massifs ou en production sur du long terme.

In [ ]:
"""# TEST essai/rendu streamlit

import joblib
import os

# Créer le dossier s'il n'existe pas
os.makedirs("model", exist_ok=True)

# Sauvegarder le modèle, les mappings et la matrice CSR
joblib.dump(als_model, "model/als_model.pkl")
joblib.dump((user_map, item_map, {v: k for k, v in item_map.items()}), "model/mappings.pkl")
joblib.dump(csr_train, "model/csr_train.pkl")"""

### item-to-item (content_id - articles similaires)

In [ ]:
# Sélectionnez un article existant
article_ref = df_articles["article_id"].iloc[0]
print("Article de référence :", article_ref)

# Récupérer les 5 plus similaires
similaires = fc.recommander_similaire_article(
    article_id=article_ref,
    df_articles=df_articles,
    similarity_matrix=similarity_matrix,
    top_n=5
)
print(f"Top 5 similaires à l'article {article_ref} :", similaires)

In [ ]:
for aid in df_articles["article_id"].sample(3, random_state=42):
    print(f"\n→ Similaires à {aid} :",
        fc.recommander_similaire_article(aid, df_articles, similarity_matrix, top_n=5))

#### Avec Surprise (SVD)

In [ ]:
# --- 1. Item–Item “collaboratif” avec SVD ---
raw_ids_svd, sim_svd = fc.build_item_similarity_svd(modele_svd, trainset)

# Choix d’un article de référence (par ex. le 1er de df_articles)
article_ref = df_articles["article_id"].iloc[0]
print("Article de référence :", article_ref)

svd_neighbors = fc.recommender_latent_item_svd(
    article_id=article_ref,
    raw_ids=raw_ids_svd,
    sim_mat=sim_svd,
    top_n=5
)
print("SVD – articles similaires :", svd_neighbors)

#### Avec Implicit (ALS)

In [ ]:
ids_als, sim_als = fc.build_item_similarity_als(als_model, item_map)

als_neighbors = fc.recommender_latent_item_als(
    article_id=article_ref,
    ids=ids_als,
    sim_mat=sim_als,
    top_n=5
)
print("ALS – articles similaires :", als_neighbors)

In [ ]:
# 3 exemples au hasard
for aid in df_articles["article_id"].sample(3, random_state=42):
    voisins = fc.recommender_latent_item_als(
        article_id=aid, ids=ids_als, sim_mat=sim_als, top_n=5
    )
    print(f"Article {aid} → similaires ALS : {voisins}")

In [ ]:
for aid in [article_ref, df_articles["article_id"].sample(2, random_state=1).iloc[1]]:
    cb = fc.recommander_similaire_article(aid, df_articles, similarity_matrix, top_n=5)
    als = fc.recommender_latent_item_als(aid, ids_als, sim_als, top_n=5)
    print(f"\nPour l’article {aid} :\n • Content-based → {cb}\n • ALS-CF       → {als}")

In [ ]:
# --- 1) Construction de la similarité hybride ---
alpha = 0.7  # 70% content-based, 30% collaborative
ids_h, sim_h = fc.build_item_similarity_hybrid(
    df_articles=df_articles,
    content_sim=similarity_matrix,
    cf_ids=ids_als,
    cf_sim=sim_als,
    alpha=alpha
)

# --- 2) Test pour un article de référence ---
article_ref = df_articles["article_id"].iloc[0]
hybrid_neighbors = fc.recommender_hybrid_article(
    article_id=article_ref,
    ids_common=ids_h,
    hybrid_sim=sim_h,
    top_n=5
)

print(f"Top 5 hybrides pour l’article {article_ref} :", hybrid_neighbors)


# --- 3) Comparaison qualitative ---
cb   = fc.recommander_similaire_article(article_ref, df_articles, similarity_matrix, top_n=5)
als_ = fc.recommender_latent_item_als(article_ref, ids_als, sim_als, top_n=5)
print(f"\nComparaison pour {article_ref}:")
print(" • Content-based →", cb)
print(" • ALS-CF        →", als_)
print(" • Hybride       →", hybrid_neighbors)


In [ ]:
# ╔════════════════════════════════════════════════════════════╗
# ║  Sauvegarde des artefacts pour la Function Azure           ║
# ╚════════════════════════════════════════════════════════════╝
from pathlib import Path
import joblib, scipy.sparse as sp

OUT_DIR = Path("azure_function/model")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# 1) Modèle collaboratif
joblib.dump(als_model, OUT_DIR / "als_model.pkl")

# 2) Mappings
reverse_item_map = {v: k for k, v in item_map.items()}
joblib.dump((user_map, item_map, reverse_item_map),
            OUT_DIR / "mappings.pkl")

# 3) Matrice interactions
sp.save_npz(OUT_DIR / "csr_train.npz", csr_train)

# 4) Top‑20 articles populaires (cold‑start & complément)
popular_items = (df_train["click_article_id"]
                .value_counts()
                .head(20)
                .index
                .tolist())
joblib.dump(popular_items, OUT_DIR / "popular_items.pkl")

print("✅  Artefacts enregistrés dans :", OUT_DIR.resolve())

À quoi sert chaque fichier ?

- als_model.pkl : le « cerveau » du moteur de reco.

- mappings.pkl : dictionnaires de traduction user_id⇆index, item_id⇆index.

- csr_train.npz : permet de filtrer les articles déjà lus lors de la recommandation.

- popular_items.pkl (facultatif) : suggestions de repli pour un nouvel utilisateur.

In [ ]:
# Test de rechargement rapide – garantit toujours 5 recommandations
import joblib, scipy.sparse as sp
from fonctions import recommander_implicit

# Chargement des artefacts
mdl   = joblib.load("azure_function/model/als_model.pkl")
u_map, i_map, _ = joblib.load("azure_function/model/mappings.pkl")
mat   = sp.load_npz("azure_function/model/csr_train.npz")
popular = joblib.load("azure_function/model/popular_items.pkl")

# Choix d'un utilisateur
some_user = next(iter(u_map.keys()))

# 1) Recommandations ALS “pures”
recs = recommander_implicit(some_user, None, mdl, u_map, i_map, mat, top_n=5)

# 2) Complément si < 5 → on pioche dans les articles populaires
if len(recs) < 5:
    recs += [a for a in popular if a not in recs][:5 - len(recs)]

print(f"Reco (5 articles) pour l'utilisateur {some_user} → {recs}")

In [ ]:
import azure.functions, joblib, implicit, scipy, numpy, pandas
print("Imports OK")